# EPL 23/34 Player Dataset

This notebook is created for study and as part of a personal project related to the English Premier League (EPL).

## Overview

The dataset provided in this notebook includes information on players who participated in EPL 23/34. The data is formatted as an SQL file for easy integration into databases.

## Data Source

The data is collected from the official [Premier League website](https://www.premierleague.com/), a reliable source for up-to-date information on EPL teams and players.

## Purpose

The primary goals of this notebook are:

1. **Study**: Understand and analyze player data in the context of the EPL 23/34 season.
2. **Personal Project**: Contribute to a personal project related to the English Premier League.

## Contents

- **Data Collection**: Player information is scraped from the Premier League website, including details such as name, position, date of birth, and team affiliation.
- **SQL Format**: The collected data is formatted into SQL queries for easy integration into database systems.

## Instructions

1. Execute the notebook to collect player data.
2. Review and analyze the SQL queries generated.
3. Integrate the SQL file into your preferred database for further analysis.

Feel free to explore, modify, and adapt the code to suit your specific needs.

---

*Note: This notebook is for educational and personal project purposes only. All data is sourced from the official Premier League website.*



In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [ ]:
log = pd.DataFrame(pd.read_csv("/kaggle/input/fantasy-premier-league-dataset-2023-2024/players.csv"))
log.sample(5)

# **Team in 23/24**

In [ ]:
team = log["team"].values
All_team = np.unique(team)
All_team

# **Team Stadium**

In [ ]:
text = open("/kaggle/input/team-stadium/Team_Stadium.txt").read()
# Split the string into lines
# lines = teams_and_stadiums.strip().split('\n')

# # Create the mapping dictionary
# # lines.remove('')
# mapping_dict = dict(zip(lines[0::2], lines[1::2]))

# # Print the mapping dictionary
# print(mapping_dict)

lines = [line.strip() for line in text.strip().split('\n') if line.strip()]
team_stadium_list = [(lines[i], lines[i + 1]) for i in range(0, len(lines), 2)]
team_stadium = np.array(team_stadium_list)

team_stadium[:5]

# **Get only played Team Stadium**

In [ ]:
# Drop unplayed team
df = pd.DataFrame(team_stadium)
# df = np.delete(df,3)
df = df.drop([2,3,4,5,6,8,12,13,15,17,20,21,22,23,24,29,31,33,34,35,36,38,39,40,41,42,43,45,46,48,49])
df

In [ ]:
# Turn in to array
df = np.array(df)
print(df[:5])

# **Scraping player data**

# Get all player

In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to extract team ID from the given URL
def extract_team_id(url):
    # Extract team ID from the URL
    team_id = url.split('/')[2]
    return team_id

# URL of the page containing club information
url = "https://www.premierleague.com/clubs"

# Fetch the content of the page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract team IDs from each club card
club_cards = soup.find_all('li', class_='club-card-wrapper')
team_ids = [extract_team_id(club_card.find('a')['href']) for club_card in club_cards]

# Display the list of team IDs
team_info = np.column_stack((df, team_ids))
team_info

In [ ]:
import bs4
import requests
from datetime import datetime

# Create a file to write the SQL queries
with open('/kaggle/working/insert_player.sql', 'a') as file:
    # Iterate through teams
    for i in range(0, team_info.shape[0]):
        url = f"https://www.premierleague.com/clubs/{team_info[i][2]}/club/squad?se=578"
        response = requests.get(url)
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        pl = []

        # Find all player information cards
        player_cards = soup.find_all('li', class_='stats-card')

        # Function to extract date of birth from player's overview page and convert it to the desired format
        def get_date_of_birth(player_url):
            player_page = requests.get(f"https://www.premierleague.com{player_url}")
            player_soup = bs4.BeautifulSoup(player_page.text, 'html.parser')

            # Find the div with label "Date of Birth" and extract the date
            dob_element = player_soup.find('div', {'class': 'player-overview__label'}, string='Date of Birth')
            if dob_element:
                dob_text = dob_element.find_next('div', {'class': 'player-overview__info'}).get_text(strip=True)
                # Parse the date string and format it
                dob_date = datetime.strptime(dob_text.split('(')[0].strip(), '%d/%m/%Y').strftime('%Y-%m-%d')
            else:
                dob_date = ''

            return dob_date

        # Iterate through each player card and extract information
        for card in player_cards:
            name_first_element = card.select_one('.stats-card__player-first')
            name_last_element = card.select_one('.stats-card__player-last')
            position_element = card.select_one('.stats-card__player-position')
            squad_number_element = card.select_one('.stats-card__squad-number')
            player_url = card.select_one('.stats-card__wrapper')['href']

            # Check if elements exist before extracting text
            name_first = name_first_element.get_text(strip=True) if name_first_element else ''
            name_last = name_last_element.get_text(strip=True) if name_last_element else ''
            position = position_element.get_text(strip=True) if position_element else ''
            squad_number = squad_number_element.get_text(strip=True) if squad_number_element else ''

            # Escape single quotes in names
            name_first = name_first.replace("'", "''")
            name_last = name_last.replace("'", "''")

            # Extract date of birth using the function
            dob = get_date_of_birth(player_url)

            pl.append([name_first, name_last, position, squad_number, dob])

        # SQL Insertion
        # Team ID
        team_id = i + 1

        # Generate a single SQL INSERT statement for all players
        sql_query = (
            "INSERT INTO `player` "
            "(`firstName`, `middleName`, `lastName`, "
            "`shirtNo`, `birthDate`, `position`, `isBanned`, "
            "`isInjured`, `teamID`)\n"
            "VALUES "
        )

        # Add values for each player
        for player_info in pl:
            sql_query += "('{}', NULL, '{}', '{}', '{}', '{}', NULL, NULL, '{}'), ".format(
                player_info[0], player_info[1], player_info[3], player_info[4], player_info[2], team_id
            )

        # Remove the trailing comma and write the SQL query to the file
        sql_query = sql_query.rstrip(', ')
        file.write(sql_query + ';\n')
#         print(sql_query)
